In [54]:
import pandas as pd
df= pd.read_csv('/workspaces/poetry-demo-test2/clean/merge_no_no_stid.csv')
df.head(1)

,nm_name,st_name,nm_city,st_address,user_name,user_id,is_local_guide,review_count,photo_count,rating_star,time_num,time_unit,months_ago,content_clean,create_date,update_date,nm_id,st_id
0,逢甲夜市,辣訣-秘藏鍋物-台中逢甲店\n,台中市,西屯區文華路121之31號,施小妹,104891691568643818609,False,5.0,0.0,5.0,1,週,0.716667,CP值高 東西也不錯吃 香米飯和功夫麵可以無限續,2025-05-04,2025-05-18,FC,NaN


In [ ]:
df.describe().round(2)

In [ ]:
df.info()

In [ ]:
#檢查維度
df.shape        # (rows, columns)

In [ ]:
#欄位清單
df.columns      # Index of column names


In [ ]:
#資料型態
df.dtypes       # 各欄位 dtype (object／int／float／datetime…)


In [ ]:
#記憶體使用量
df.memory_usage(deep=True)

# 空值與唯一值分析

In [ ]:
#每欄缺失值計數
df.isnull().sum()

In [ ]:
#每欄不同值（unique）數量
df['user_id'].nunique

In [ ]:
#查看缺失值比例
df.isnull().mean()  # 轉成比例


# 取樣與重點觀察


In [ ]:
#隨機抽樣
df.sample(5)

In [ ]:
#頂端與底端
df.head(3)
df.tail(3)

In [55]:
#Value Counts（類別分佈）
df['st_name'].value_counts().head(20)   # 單欄位
#df['matched_name'].value_counts().head(10)
#計算 matched_name（店家名稱）欄位中，每家店出現的評論數量，並取出前 10 名。

st_name
🈺福客素食蛋餅早餐——原素食蛋餅（蛋奶素） 早午餐 —漢堡，吐司，炒麵，抓餅，蘿蔔糕，炸物，飲料均有販售，歡迎訂購    127
辣訣-秘藏鍋物-台中逢甲店\n                                              120
古早味糖葫蘆                                                         3
Name: count, dtype: int64

In [ ]:
df = df.drop(df[df['st_name'] == '大東東夜市'].index)

In [58]:
mark = (df['st_name'] == '古早味糖葫蘆')
df.loc[mark, 'st_id'] = 2036

In [ ]:
mark = df['st_name'] == '刁四麻辣燙（士林夜市店）'
df.loc[mark, 'st_name'] = '刁四麻辣燙'
df

# 文字欄位探索 


In [ ]:
#最長\最短字串
df['content_clean'].str.len().describe()


In [ ]:
#關鍵詞出現次數
df['content_clean'].str.contains('難吃').sum()


In [ ]:
#前幾個常用詞（需先做斷詞／split）
from collections import Counter
cnt = Counter(" ".join(df['content_clean'].dropna()).split())
cnt.most_common(10)

# 重複資料

In [ ]:
# 1. 計算每位作者發文次數
author_counts = df['user_id'].value_counts()

# 2. 篩出發文次數多於 1 的作者
repeat_authors = author_counts[author_counts > 1]
print(repeat_authors.head(10))
# 這裡 .head(10) 只是示意，會顯示發文次數最多的前 10 位重複作者

# 3. 要取得整個名單
all_repeat = repeat_authors.index.tolist()
print(f"一共 {len(all_repeat)} 位作者發表超過一次評論")

# 4. 把這些作者在原 df 中的所有留言都抓出來
df_repeat = df[df['user_id'].isin(all_repeat)]
df_repeat.head()


In [ ]:
# 直接標記 評論內容 欄的重複列（keep=False 代表所有重複都標 True）
df_dup2 = df[df.duplicated(subset=['st_address', 'st_name_x'], keep=False)]

# 同樣依 content → author 排序並檢視
df_dup = df_dup2.sort_values(['st_address','st_id','st_name_x','st_name','st_score', 'st_total_com'])[['st_address','st_id','st_name_x','st_name','st_score', 'st_total_com']]


In [ ]:
df_dup

In [ ]:
df_dup[df_dup["st_name"] != df_dup["st_name_x"]]

In [ ]:
df = df[df["st_name"] != df["st_name_x"]]

In [ ]:
dup_cols = ['user_id', 'nm_id', 'st_id', 'rating_star', 'content_clean']

# groupby 然後 filter 出出現次數 > 2 的群組
df_repeated = df_dup2.groupby(dup_cols).filter(lambda x: len(x) > 2)
df_repeat = df_repeated.sort_values(['user_id','nm_id','st_id', 'rating_star', 'content_clean'])
# dup_counts = df_dup2.groupby(dup_cols).size().reset_index(name='count')
# dup_over_2 = dup_counts[dup_counts['count'] > 2]

In [ ]:
df_repeat

In [ ]:
df = df.drop(columns=['st_id_x', 'st_id_y'])

In [ ]:
df['nm_name'] = df['nm_name_y']
df['nm_city'] = df['nm_city_y']
df['st_address'] = df['st_address_y']
df = df.drop(columns=['nm_name_y', 'nm_city_y', 'st_address_y'])

In [ ]:
df

In [ ]:
df = df.drop_duplicates(subset=['comment_id'], keep='last')

In [ ]:
df_no_stid = df[df['st_id'].isna() |(df['st_id'] == '')]

In [ ]:
df = df.drop(df[df['st_id'].isna() |(df['st_id'] == '')].index)

In [ ]:
df

In [ ]:
df = df.drop(columns=['st_name_y'])

In [ ]:
df

In [ ]:
mark = df['st_address_y'].isna() |(df['st_address_y'] == '')
df.loc[mark, 'st_address_y'] = df.loc[mark, 'st_address_x']

In [ ]:
df['st_address_x'] = df['st_address_y'].where(df['st_address_y'].notna(), df['st_address_x'])

In [ ]:
df_repeat.to_csv("/workspaces/poetry-demo-test2/toDB/comment_dup_3.csv", header=True, index=None)

In [ ]:
df_dup.to_csv("/workspaces/poetry-demo-test2/store/test2.csv", header=True, index=None)

# 個人驗證

In [ ]:
author = '102322134975713908116'
df_author = df[df['user_id'] == author]
df_author

In [ ]:
#留言時間
df_author[f'{time_num}+{time_unit}'].value_counts()

In [ ]:
#對「評論內容」欄，標記出哪些列的值和前面某一列完全一樣
#預設 keep='first'，所以第一筆出現的那個不算重複，從第二筆開始才算
#把這些標記為重複（True）的列數加總起來，得到重複評論的筆數。
df_author['content_clean'].duplicated().sum()

In [ ]:
df_nodup = df[~(df['fake_comment'] == '無法判斷')]

In [59]:
df.to_csv("/workspaces/poetry-demo-test2/clean/merge_no_no_stid.csv", header=True, index=None)